## Step 1: Training Setup

### Task #1

Setting the following variables

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math
import nltk
nltk.download('punkt')

batch_size = 128         # batch size
vocab_threshold = 7        # minimum word count threshold
vocab_from_file = True   # if True, load existing vocab file
embed_size = 256           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 3             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# writing image transformations
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())
          
# Define the optimizer.
optimizer = torch.optim.Adam(params, lr=0.001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.04s)
creating index...


  0%|          | 759/414113 [00:00<01:54, 3618.83it/s]

index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [01:34<00:00, 4373.05it/s]


## Step 2: Train your Model

In [5]:
from workspace_utils import active_session
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# Open the training log file.
f = open(log_file, 'w')

with active_session():
    for epoch in range(1, num_epochs+1):

        for i_step in range(1, total_step+1):

            # Randomly sample a caption length, and sample indices with that length.
            indices = data_loader.dataset.get_train_indices()
            # Create and assign a batch sampler to retrieve a batch with the sampled indices.
            new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
            data_loader.batch_sampler.sampler = new_sampler

            # Obtain the batch.
            images, captions = next(iter(data_loader))

            # Move batch of images and captions to GPU if CUDA is available.
            images = images.to(device)
            captions = captions.to(device)

            # Zero the gradients.
            decoder.zero_grad()
            encoder.zero_grad()

            # Pass the inputs through the CNN-RNN model.
            features = encoder(images)
            outputs = decoder(features, captions)

            # Calculate the batch loss.
            loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))

            # Backward pass.
            loss.backward()

            # Update the parameters in the optimizer.
            optimizer.step()

            # Get training statistics.
            stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))

            # Print training statistics (on same line).
            print('\r' + stats, end="")
            sys.stdout.flush()

            # Print training statistics to file.
            f.write(stats + '\n')
            f.flush()

            # Print training statistics (on different line).
            if i_step % print_every == 0:
                print('\r' + stats)

        # Save the weights.
        if epoch % save_every == 0:
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

Epoch [1/3], Step [100/3236], Loss: 2.0393, Perplexity: 7.6855
Epoch [1/3], Step [200/3236], Loss: 2.0037, Perplexity: 7.41624
Epoch [1/3], Step [300/3236], Loss: 2.0837, Perplexity: 8.03388
Epoch [1/3], Step [400/3236], Loss: 2.2009, Perplexity: 9.03354
Epoch [1/3], Step [500/3236], Loss: 2.1110, Perplexity: 8.25680
Epoch [1/3], Step [600/3236], Loss: 2.1678, Perplexity: 8.73933
Epoch [1/3], Step [700/3236], Loss: 2.0030, Perplexity: 7.41118
Epoch [1/3], Step [800/3236], Loss: 2.1789, Perplexity: 8.83651
Epoch [1/3], Step [900/3236], Loss: 2.3058, Perplexity: 10.0317
Epoch [1/3], Step [1000/3236], Loss: 1.9142, Perplexity: 6.7816
Epoch [1/3], Step [1100/3236], Loss: 1.9716, Perplexity: 7.18253
Epoch [1/3], Step [1200/3236], Loss: 2.0434, Perplexity: 7.71652
Epoch [1/3], Step [1300/3236], Loss: 1.9894, Perplexity: 7.31139
Epoch [1/3], Step [1400/3236], Loss: 2.3539, Perplexity: 10.5265
Epoch [1/3], Step [1500/3236], Loss: 1.9957, Perplexity: 7.35769
Epoch [1/3], Step [1600/3236], Loss: